# Neuromodulation example  (bath)

This neuromodulation creates a small network of neurons. We also use the reaction diffusion model by Anu G Nair 2015.

To generate the ```reaction_diffusion_D1.json``` file in ```data/str-dspn-e150602_c1_D1-mWT-0728MSN01-v20211026/``` from the xml file we run ```data/convert_sbml_to_json.sh```

Reference for DA modulation cascade:

Lindroos R, Dorst MC, Du K, et al. Basal Ganglia Neuromodulation Over Multiple Temporal and Structural Scales-Simulations of Direct Pathway MSNs Investigate the Fast Onset of Dopaminergic Effects and Predict the Role of Kv4.2. Front Neural Circuits. 2018;12:3. Published 2018 Feb 6. doi:10.3389/fncir.2018.00003

Here we use the data from Planert:
Planert H, Berger TK, Silberberg G. Membrane properties of striatal direct and indirect pathway neurons in mouse and rat slices and their modulation by dopamine. PLoS One. 2013;8(3):e57054. doi:10.1371/journal.pone.0057054

The neurons are held at -60mV, or -80mV, then a current injection is applied for 500ms to trigger spiking.

Figure 4:


D2 -- figure 3
https://www.jneurosci.org/content/35/41/14017/tab-figures-data
10000nm DA to get high PKA response


## Network setup

We have two neurons. The first neuron (Neuron 0) receives external input (cortical from t=0s and DA from t=0.1s). The cortical input will activate the first neuron, and through activation of synapses on the second neuron, we will see the dopamine level increase in the second neuron (Neuron 1).

The first neuron also receives direct DA activation from external input (starting at 100ms).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import plotly.io as pio 
pio.templates.default = "simple_white"

In [3]:
import os
from snudda import Snudda

neuron = "dspn"
# neuron = "ispn"

neuron_path = os.path.join("data", neuron)
network_path = os.path.join("networks", "neuromodulation_bath_current_SBML")

In [4]:
snudda = Snudda(network_path=network_path)
#si = snudda.init_tiny(neuron_paths=neuron_path, neuron_names="dspn", number_of_neurons=[1], 
#                      random_seed=123)
si = snudda.init_tiny(neuron_paths=neuron_path, neuron_names=neuron, number_of_neurons=[1], 
                      random_seed=123)

si.network_data["regions"]["Cube"]["neurons"]["dspn"]["reaction_diffusion"] = "data/JSON/reaction_diffusion_D1.json"
# si.network_data["regions"]["Cube"]["neurons"]["ispn"]["reaction_diffusion"] = "data/JSON/reaction_diffusion_D2.json-updated"


# How the ion channels are modified by DA
si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation"] = "test-modulation.json"
# si.network_data["regions"]["Cube"]["neurons"]["dspn"]["modulation"] = "test-modulation-no-effect.json"
# si.network_data["regions"]["Cube"]["neurons"]["dspn"]["modulation"] = "test-modulation-exclude-all.json"


si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation_key"] = "abc"

si.write_json()

snudda.create_network()

Adding neurons: dspn from dir data/dspn
Writing networks/neuromodulation_bath_current_SBML/network-config.json
Writing networks/neuromodulation_bath_current_SBML/network-config.json
Placing neurons
Network path: networks/neuromodulation_bath_current_SBML
Reading SNUDDA_DATA=None from networks/neuromodulation_bath_current_SBML/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/utils/../data from networks/neuromodulation_bath_current_SBML/network-synapses.hdf5
No n_putative_points and putative_density, setting n_putative_points = 46
(this must be larger than the number of neurons you want to place)
Generating 46 points for networks/neuromodulation_bath_current_SBML/mesh/Cube-cube-mesh-2.3159794767993218e-05.obj
Filtering, keeping inside points: 1 / 20
Managed to create 1 putative points within the volume.
  WARNING --> is the volume too small? You can create new cube mesh using create_cube_mesh.py

Example how to use create_cube_mesh.py:
from snudda.place.create_cube_

In [5]:
# Free memory
snudda = None

### No synaptic input, only current injections!

In [6]:
sim_output_neuromodulation_ON = os.path.join(network_path, "simulation", "output_neuromodulation_ON.hdf5")
sim_output_neuromodulation_OFF = os.path.join(network_path, "simulation", "output_neuromodulation_OFF.hdf5")

In [7]:
# mech_dir = os.path.join("..", "..", "..", "..", "BasalGangliaData", "data", "neurons", "mechanisms")
mech_dir = "/home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms"
sample_dt = None # 0.00005

# sim = snudda.simulate(time=0, mech_dir=mech_dir, verbose=True, sample_dt=sample_dt, output_file=sim_output_neuromodulation_ON)

In [8]:
# sim_config_on = os.path.join("data", "da_experiment_on.json")
# sim_config_off = os.path.join("data", "da_experiment_off.json")

snudda = None

sim_config_on = os.path.join("data", "da_experiment_cur_inj_on_bath_sbml.json")
sim_config_off = os.path.join("data", "da_experiment_cur_inj_off_bath_sbml.json")


sim_time = 5
n_workers = 2

## Running simulations

To see progress of the two simulations in log files ```networks/neuromodulation_ON_OFF/log/network-simulation-ON.txt``` and ```networks/neuromodulation_ON_OFF/log/network-simulation-OFF.txt```.

In [9]:
run_str_on = f"snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_on} --mechdir {mech_dir} --enable_rxd_neuromodulation"
print(run_str_on)

snudda simulate networks/neuromodulation_bath_current_SBML --time 5 --simulation_config data/da_experiment_cur_inj_on_bath_sbml.json --mechdir /home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms --enable_rxd_neuromodulation


In [11]:
# %%timeit
os.system(run_str_on)

numprocs=1
args.ipython_profile = None
args: Namespace(action='simulate', path='networks/neuromodulation_bath_current_SBML', network_file=None, input_file=None, output_file=None, time=5.0, snudda_data=None, simulation_config='data/da_experiment_cur_inj_on_bath_sbml.json', record_volt=True, randomseed=None, disable_synapses=None, disable_gj=None, mech_dir='/home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms', profile=False, verbose=False, exportCoreNeuron=False, record_all=None, enable_rxd_neuromodulation=True, disable_rxd_neuromodulation=None, ipython_profile=None)
MPI Rank: 0, Size: 1
Using input file None
NEURON mechanisms already compiled, make sure you have the correct version of NEURON modules.
If you delete x86_64, aarch64, arm64 directories (or nrnmech.dll) then you will force a recompilation of the modules.
Reading SNUDDA_DATA=None from networks/neuromodulation_bath_current_SBML/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/utils/../data from netw

Traceback (most recent call last):
  File "/home/hjorth/HBP/Snudda/venv/bin/snudda", line 8, in <module>
    sys.exit(snudda_cli())
  File "/home/hjorth/HBP/Snudda/snudda/cli.py", line 211, in snudda_cli
    actions[args.action](args)
  File "/home/hjorth/HBP/Snudda/snudda/core.py", line 748, in simulate_wrapper
    sim = self.simulate(network_file=args.network_file, input_file=args.input_file,
  File "/home/hjorth/HBP/Snudda/snudda/core.py", line 866, in simulate
    sim.setup()
  File "/home/hjorth/HBP/Snudda/snudda/simulate/simulate.py", line 360, in setup
    self.setup_neurons()
  File "/home/hjorth/HBP/Snudda/snudda/simulate/simulate.py", line 952, in setup_neurons
    self.neurons[ID].instantiate(sim=self.sim, extracellular_regions=self.extracellular_regions)
  File "/home/hjorth/HBP/Snudda/snudda/neurons/neuron_model_extended.py", line 502, in instantiate
    param.instantiate(sim=sim, icell=self.icell)
  File "/home/hjorth/HBP/Snudda/venv/lib/python3.9/site-packages/bluepyopt/

256

### Rerun without neuromodulation

In [12]:
run_str_off = f"snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_off} --mechdir {mech_dir} "
print(run_str_off)

snudda simulate networks/neuromodulation_bath_current_SBML --time 5 --simulation_config data/da_experiment_cur_inj_off_bath_sbml.json --mechdir /home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms 


In [13]:
os.system(run_str_off)

numprocs=1
args.ipython_profile = None
args: Namespace(action='simulate', path='networks/neuromodulation_bath_current_SBML', network_file=None, input_file=None, output_file=None, time=5.0, snudda_data=None, simulation_config='data/da_experiment_cur_inj_off_bath_sbml.json', record_volt=True, randomseed=None, disable_synapses=None, disable_gj=None, mech_dir='/home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms', profile=False, verbose=False, exportCoreNeuron=False, record_all=None, enable_rxd_neuromodulation=None, disable_rxd_neuromodulation=None, ipython_profile=None)
MPI Rank: 0, Size: 1
Using input file None
NEURON mechanisms already compiled, make sure you have the correct version of NEURON modules.
If you delete x86_64, aarch64, arm64 directories (or nrnmech.dll) then you will force a recompilation of the modules.
Reading SNUDDA_DATA=None from networks/neuromodulation_bath_current_SBML/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/utils/../data from net

0

## Load the data and plot

In [14]:
from snudda.utils import SnuddaLoadSimulation

nd = SnuddaLoadSimulation(sim_output_neuromodulation_ON)
time = nd.get_time()
data_pka = nd.get_data("PKAc", 0)[0][0]
data_da = nd.get_data("DA", 0)[0][0]
data_da_external = nd.get_data("DA", 0)[0][0]

# This is saved with add_rxd_internal_concentration_recording_all -- check that it worked 
data_pka_all0 = nd.get_data("PKAc", 0)[0][0]

Loading networks/neuromodulation_bath_current_SBML/simulation/output_neuromodulation_ON.hdf5
WARNING. Depolarisation block in neuron - neuron_id: (name, parameter_key, morphology_key):
0: (dspn_0, pe6ec2d4b, mf702205f)


In [15]:
nd.list_data_types(0)

['AC5',
 'AC5GaolfGTP',
 'AC5GaolfGTP_ATP',
 'AC5_ATP',
 'AMP',
 'ATP',
 'D1R',
 'D1RDA',
 'D1RDAGolf',
 'D1RGolf',
 'DA',
 'GaolfGDP',
 'GaolfGTP',
 'Gbgolf',
 'Golf',
 'PDE10',
 'PDE10_cAMP',
 'PDE10c',
 'PDE10c_cAMP',
 'PDE4',
 'PDE4_cAMP',
 'PKA',
 'PKAc',
 'PKAcAMP2',
 'PKAcAMP4',
 'PKAreg',
 'cAMP',
 'cal12_ms.modulation_factor',
 'cal13_ms.modulation_factor',
 'kaf_ms.modulation_factor_g',
 'kaf_ms.modulation_factor_shift',
 'kas_ms.modulation_factor',
 'kir_ms.modulation_factor',
 'naf_ms.modulation_factor',
 'spikes',
 'voltage']

In [16]:
data_types = nd.list_data_types(0)
all_species_data = nd.get_all_data(neuron_id=0, exclude=["spikes", "voltage"])
time = nd.get_time()
voltage = nd.get_data("voltage", [0])

In [17]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in all_species_data:
    idx = time >= 0.0
    fig.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type, line={"width":4}))

"""
fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800,
                 font={"size":18},  # General font size for all elements
                 legend={"font":{"size":16}},  # Specific font size for legend
                 xaxis={"title":{"font":{"size":20}}, "tickfont":{"size":14}},  # X-axis title and tick labels
                 yaxis={"title":{"font":{"size":20}}, "tickfont":{"size":14}})   # Y-axis title and tick labels
"""
fig.show()

In [ ]:
# Reporting plot

import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in ["D1RDA", "PKAc", "cAMP"]: #all_species_data:
    idx = time >= 0.0
    fig.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type, line={"width":4}))

fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}})   # Y-axis title and tick labels


fig.show()

fig.write_image("example-da-cascade-1.png", width=1200, height=800)


fig2 = go.Figure()
for data_type in ["cal12_ms.modulation_factor", "cal13_ms.modulation_factor", 
                  "kas_ms.modulation_factor", "kir_ms.modulation_factor", "naf_ms.modulation_factor", ]: #all_species_data:
    idx = time >= 0.0
    data_type_str = data_type.replace("modulation_factor", "modulation")
    fig2.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type_str, line={"width":4}))

fig2.update_layout(xaxis_title="Time (s)", yaxis_title="Modulation factor", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}})   # Y-axis title and tick labels

fig2.show()
fig2.write_image("example-da-cascade-2-effect.png", width=1200, height=800)


In [ ]:
# Reporting plot

import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in ["D2RDA", "PKAc", "cAMP"]: #all_species_data:
    idx = time >= 0.0
    fig.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type, line={"width":4}))

fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}})   # Y-axis title and tick labels


fig.show()

fig.write_image("example-da-cascade-1.png", width=1200, height=800)


fig2 = go.Figure()
for data_type in ["cal12_ms.modulation_factor", "cal13_ms.modulation_factor", 
                  "kas_ms.modulation_factor", "kir_ms.modulation_factor", "naf_ms.modulation_factor", ]: #all_species_data:
    idx = time >= 0.0
    data_type_str = data_type.replace("modulation_factor", "modulation")
    fig2.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type_str, line={"width":4}))

fig2.update_layout(xaxis_title="Time (s)", yaxis_title="Modulation factor", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}})   # Y-axis title and tick labels

fig2.show()
fig2.write_image("example-da-cascade-2-effect.png", width=1200, height=800)


In [ ]:
# fig.write_html("ask-jeanette2.html")

In [ ]:
data_types2 = nd.list_data_types(0)
all_species_data2 = nd.get_all_data(neuron_id=1, exclude=["spikes", "voltage"])
time = nd.get_time()
voltage = nd.get_data("voltage", [0, 1])

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in all_species_data2:
    fig.add_trace(go.Scatter(x=time, y=all_species_data2[data_type][0][1].T[0], name=data_type))

fig.update_layout(title="With DA modulation", xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800)
fig.show()
fig.write_image("DA_modulation_reactions.png", width=1200, height=800)


In [ ]:
nd_off = SnuddaLoadSimulation(sim_output_neuromodulation_OFF)
time_off = nd_off.get_time()
voltage_off = nd_off.get_data("voltage", [0, 1])

In [ ]:
fig = go.Figure()

sct_on = go.Scatter(x=time, y=voltage[0][0][:,0], name="DA", opacity=0.5,  line={"width":4, "color":"red"})
sct_off = go.Scatter(x=time_off, y=voltage_off[0][0][:,0], name="No DA", opacity=0.5,  line={"width":4, "color":"black"})
fig.add_traces([sct_on, sct_off])

fig.update_layout(xaxis_title="Time (s)", yaxis_title="Voltage (V)", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}, "range":[1,2.5]},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}})   # Y-axis title and tick labels
fig.show()
fig.write_image("example-trace.png", scale=2, height=800, width=1200)

In [ ]:
fig = go.Figure()

sct_on = go.Scatter(x=time, y=voltage[0][1][:,0], name="DA", opacity=0.5,  line={"width":4, "color":"red"})
sct_off = go.Scatter(x=time_off, y=voltage_off[0][1][:,0], name="No DA", opacity=0.5,  line={"width":4, "color":"black"})
fig.add_traces([sct_on, sct_off])

fig.update_layout(xaxis_title="Time (s)", yaxis_title="Voltage (V)", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}, "range":[1,2.5]},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}})   # Y-axis title and tick labels

fig.show()
fig.write_image("example-trace2.png", scale=2, height=800, width=1200)

## Plotting simulation with DA modulation 

DA is active from 0.3 to 1.3 seconds

In [ ]:
%matplotlib inline

from snudda.plotting.plot_traces import PlotTraces
pt = PlotTraces(output_file=sim_output_neuromodulation_ON)
# Use trace_id to specify which traces
ax = pt.plot_traces(offset=0, time_range=None,fig_size=(10,4))

## Plot simulation, with neuromodulation disabled

In [ ]:
%matplotlib inline

from snudda.plotting.plot_traces import PlotTraces
pt_off = PlotTraces(output_file=sim_output_neuromodulation_OFF)
# Use trace_id to specify which traces
ax_off = pt_off.plot_traces(offset=0, time_range=None,fig_size=(10,4))

# Extracting steady state values

Extract steady state values: 
```python ../../../snudda/utils/extract_steady_state_values.py data/JSON/reaction_diffusion_D1_from_SBTab.json networks/neuromodulation_bath_current_SBTab/simulation/output_neuromodulation_ON.hdf5```

Verify that the rates are correct, the rename the JSON file ```reaction_diffusion_D1_from_SBTab.json-updated``` to ```reaction_diffusion_D1_from_SBTab.json```

In [ ]:
time_off = nd_off.get_time()
data_types_off = nd_off.list_data_types(0)
all_species_data_off = nd_off.get_all_data(neuron_id=0, exclude=["spikes", "voltage"])


In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in all_species_data_off:
    idx = time >= 0.0
    fig.add_trace(go.Scatter(x=time_off[idx], y=all_species_data_off[data_type][0][0].T[0][idx], name=data_type))

fig.update_layout(title="No DA modulation", xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800)
fig.show()

In [ ]:
from snudda.plotting.plotly.trace import PlotTrace

pt_on = PlotTrace(snudda_load_simulation=nd)
pt_on.define_colour_by_neuron_id({0: "blue", 1: "red"})

pt_on.plot_traces()

In [ ]:
from snudda.plotting.plotly.trace import PlotTrace

pt_off = PlotTrace(snudda_load_simulation=nd_off)
pt_off.define_colour_by_neuron_id({0: "blue", 1: "red"})

pt_off.plot_traces()